In [2]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path = "../data/drinks.csv"
drinks = pd.read_csv(file_path)

# 결측치 대체
drinks['continent'] = drinks['continent'].fillna('OT')

In [15]:
# 대륙별 평균 wine_serving 탐색
drinks["wine_servings_cont_avg"] = drinks.groupby("continent")["wine_servings"].mean()
drinks[["country", "continent", "wine_servings_cont_avg"]].sample(5).head()

,country,continent,wine_servings_cont_avg
123,Malta,EU,NaN
95,Vietnam,AS,NaN
130,Moldova,EU,NaN
178,St. Vincent & the Grenadines,OT,NaN
83,Russian Federation,AS,NaN


In [17]:
drinks['total_servings'] = drinks['beer_servings'] + drinks['spirit_servings'] + drinks['wine_servings']
print(drinks['total_servings'])

0       39
1      319
2       51
3      243
4       39
      ... 
188    404
189    344
190    313
191    370
192    436
Name: total_servings, Length: 193, dtype: int64


In [19]:
# 전체 평균보다 적은 알코올을 섭취하는 대륙 구하기
total_mean = drinks.total_litres_of_pure_alcohol.mean()
continent_mean = drinks.groupby("continent")["total_litres_of_pure_alcohol"].mean()
continent_under_mean = continent_mean[continent_mean <= total_mean].index.tolist()
df_continent_under_mean = drinks.loc[drinks.continent.isin(continent_under_mean)]

most_spirit_under_mean = df_continent_under_mean.loc[
    df_continent_under_mean["spirit_servings"].idxmax()
]
# 결과를 출력합니다.
most_spirit_under_mean["country"]

'Russian Federation'

In [20]:
# 술 소비량 대비 알코올 비율
# 술 소비량 대비 알콜 비율에 대한 칼럼을 만들어서 병합합니다.
drinks["alcohol_rate"] = (
    drinks["total_litres_of_pure_alcohol"] / drinks["total_servings"]
)
drinks["alcohol_rate"] = drinks["alcohol_rate"].fillna(0)

print(drinks["alcohol_rate"])

0      0.017949
1      0.018495
2      0.021569
3      0.022222
4      0.110256
         ...   
188    0.018069
189    0.017733
190    0.017891
191    0.017838
192    0.017661
Name: alcohol_rate, Length: 193, dtype: float64


In [24]:
# 대륙별 술 소비량 대비 알콜 비율.
continent_sum = drinks.groupby("continent").sum()
continent_sum["alcohol_rate_continent"] = (
    continent_sum["total_litres_of_pure_alcohol"] / continent_sum["total_servings"]
)
continent_sum = continent_sum.reset_index()
continent_sum = continent_sum[["continent", "alcohol_rate_continent"]]

drinks = pd.merge(drinks, continent_sum, on="continent", how="outer")
drinks[["country", "continent", "alcohol_rate_continent"]].sample(5).head()

,country,continent,alcohol_rate_continent
76,Myanmar,AS,0.020293
60,North Korea,AS,0.020293
192,Venezuela,SA,0.017909
11,Comoros,AF,0.031970
9,Central African Republic,AF,0.031970
